# Pipeline Research
when developing a pipeline, I use this notebook to play around with the data and see what I can do with it.

In [2]:
from datasets.documents_dataset import DocumentsDataset
from datasets.stock_dataset import StockMeta
from pipeline.document_filterer import DefaultFilterer

doc_dataset = DocumentsDataset(documents_csv_path="./organized_data/documents.csv")
filterer = DefaultFilterer()
stock_meta = StockMeta(stock_meta_path="./organized_data/stock_metadata.csv")

In [4]:
stock_name = '台積電'

stock = stock_meta.get_stock_by_name(stock_name)
filtered_docs = filterer.filter_documents(doc_dataset, stock)

In [5]:
filtered_docs

DocumentsDataset(size=72508)